In [2]:

import glob
import tqdm
import torch
import json
import os
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import tensorboard


# import package

# model
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
from torch import optim

# dataset and transformation
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import os

# display images
from torchvision import utils
import matplotlib.pyplot as plt
%matplotlib inline

# utils
import numpy as np
from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter
import time
import copy

from model import *
from dataset import *

In [3]:
def mobilenet(alpha=2, num_classes=5):
    return MobileNet(alpha, num_classes)

decive = 'cuda'
writer = SummaryWriter()
losss = {}
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = mobilenet().to(device)

transform = transforms.Compose([transforms.ToTensor(),])
dataset = KwenDataset(path = 'dataset', transform= transform, len_wqi = 8,lstm = True, label_type = False)

dataset_size = len(dataset)
train_size = int(dataset_size * 0.8)
validation_size = int(dataset_size * 0.1)
test_size = dataset_size - train_size - validation_size
train_dataset, validation_dataset, test_dataset = random_split(dataset, [train_size, validation_size, test_size])


train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, drop_last=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=4, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=True, drop_last=True)


In [5]:
model = mobilenet().to(device)


#criterion = nn.MSELoss().to(device)
criterion = nn.CrossEntropyLoss()
learning_rate = 0.0005
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


count_idx = 0
count_idx2 = 0

model = model.train()
for epoch in tqdm.tqdm(range(1000)):
    
    cost = 0.0
    model.train()
    
    for batch_idx, batch in enumerate(train_dataloader):
        print(batch_idx)
        count_idx +=1
        
        img = batch[0]
        wqi =batch[1]
        label = batch[2]
        
        label = label.to(device)
        label = label.to(torch.int64)
        
        x_= torch.stack(wqi).to(device)
        x_ = torch.transpose(x_,0,1)
        x_ = x_.to(torch.float32)

        x = img
        x = x.to(device)
        
        data = [x, x_]
        output = model(data)
        output = output.to(torch.float32)


        #output = output.view(-1)
        
        
        
        loss = criterion(output, label)
        
        loss = loss.to(torch.float32)
        loss.backward()
        
        optimizer.step()
        #scheduler.step(loss)
        
        #print('Loss: ', loss, 'epoch: ', epoch, "lr: ", optimizer.param_groups[0]['lr'])
        #print('output: ', output[0:5])
        #print('label : ', label[0:5])
        if count_idx%30 == 1 :
            optimizer.param_groups[0]['lr'] *= 0.99
        
        if count_idx%100 ==1 and count_idx >100:

            writer.add_scalar('train_loss', loss, epoch)
            print('train:\t',loss, epoch)
            print('train:{}////{}'.format(output[:4],label[:4]))
            break

    with torch.no_grad():
        model.eval()
        for batch_v in validation_dataloader:
            count_idx2 +=1
            img = batch_v[0]
            wqi =batch_v[1]
            label = batch_v[2]

            label = label.to(torch.float32)
            
            label = label.to(device)

            x_= torch.stack(wqi).to(device)
            x_ = torch.transpose(x_,0,1)
            x_ = x_.to(torch.float32)

            x = img
            x = x.to(device)

            data = [x, x_]
            output = model(data)
            output = output.view(-1)
            
            loss = criterion(output, label)

            loss = loss.to(torch.float32)
            if count_idx2%100 ==1 and count_idx2 >100:          
                writer.add_scalar('validation_loss', loss, epoch)
                print('evaluate:\t', loss, epoch)
                break
    
torch.save(model, os.path.join('model_save', 'model.pth'))


  0%|          | 0/1000 [00:00<?, ?it/s]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83


In [43]:
nn.Softmax(output)

Softmax(
  dim=tensor([[ 8.1953e-04,  1.1585e-04, -1.6642e-04, -9.9622e-05, -9.1605e-04],
          [ 6.8341e-04,  2.9948e-04, -3.7609e-04,  7.7140e-05, -5.3084e-04],
          [ 8.8212e-04,  2.1449e-04,  1.5408e-04,  1.0190e-04, -1.2858e-03],
          [ 7.2188e-04,  1.2843e-04,  2.0496e-05,  1.1666e-04, -8.1953e-04],
          [ 8.8163e-04, -1.8742e-04, -8.4497e-05,  6.9156e-05, -7.9518e-04],
          [ 1.5685e-03, -1.7716e-03, -1.3095e-03,  1.0438e-03, -1.7935e-03],
          [ 5.5871e-04,  6.7618e-05, -4.6266e-04,  1.4725e-04, -7.3185e-04],
          [ 9.0222e-04, -9.1765e-05, -2.7312e-04,  2.1300e-04, -1.0455e-03],
          [ 8.5473e-04,  3.2693e-04, -2.7685e-04,  4.2735e-04, -6.6907e-04],
          [ 4.8543e-04,  8.9527e-05, -1.9716e-04, -3.9301e-04, -3.7348e-04],
          [ 1.1271e-03, -2.3277e-04, -4.8019e-04,  1.9916e-04, -7.0473e-04],
          [ 1.2991e-03, -7.1030e-05, -4.7138e-06,  6.7375e-06, -8.6921e-04],
          [ 6.3944e-04,  3.5273e-04, -3.2510e-05,  1.4168e-04, 

In [33]:
nn.

torch.Size([5, 16])

In [34]:
criterion(out2,label)

ValueError: Expected input batch_size (5) to match target batch_size (16).

In [7]:
model = torch.load(os.path.join('model_save', 'model.pth'))

In [8]:
model

MobileNet(
  (conv1): BasicConv2d(
    (conv): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6()
    )
  )
  (conv2): Depthwise(
    (depthwise): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6()
    )
    (pointwise): Sequential(
      (0): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6()
    )
    (seblock): SEBlock(
      (squeeze): AdaptiveAvgPool2d(output_size=(1, 1))
      (excitation): Sequential(
        (0): Linear(in_features=128, out_features=8, bias=True)
        (1): ReLU()
        (2): Linear(in_features=8, out_features=128, bias=True)

In [12]:
model.eval()

criterion = nn.MSELoss().to(device)
learning_rate = 0.0005
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


count_idx2 =0
for batch_v in validation_dataloader:
    count_idx2 +=1
    img = batch_v[0]
    wqi =batch_v[1]
    label = batch_v[2]
    label = label.to(torch.float32)
    label = label.to(device)
    x_= torch.stack(wqi).to(device)
    x_ = torch.transpose(x_,0,1)
    x_ = x_.to(torch.float32)
    x = img
    x = x.to(device)
    data = [x, x_]
    output = model(data)
    output = output.view(-1)
    
    loss = criterion(output, label)
    loss = loss.to(torch.float32)
    if count_idx2%100 ==1 and count_idx2 >100:          
        writer.add_scalar('validation_loss', loss)
        print('evaluate:\t', loss)

evaluate:	 tensor(2086.0508, device='cuda:0', grad_fn=<MseLossBackward0>)
evaluate:	 tensor(3408.9556, device='cuda:0', grad_fn=<MseLossBackward0>)
evaluate:	 tensor(3682.2661, device='cuda:0', grad_fn=<MseLossBackward0>)
evaluate:	 tensor(2658.1648, device='cuda:0', grad_fn=<MseLossBackward0>)
evaluate:	 tensor(2505.6021, device='cuda:0', grad_fn=<MseLossBackward0>)
evaluate:	 tensor(3346.3108, device='cuda:0', grad_fn=<MseLossBackward0>)
evaluate:	 tensor(2568.4443, device='cuda:0', grad_fn=<MseLossBackward0>)
evaluate:	 tensor(2367.9546, device='cuda:0', grad_fn=<MseLossBackward0>)


In [8]:
#model = RNNNet(input_size=8, num_classes=1, hidden_size = 8, init_weights=True).to(device)
device = 'cuda'
model = testNet(1).to(device)

criterion = nn.MSELoss().to(device)
learning_rate = 0.01
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
#optimizer = optim.SGD(model.parameters(),lr=learning_rate)

count_idx = 0
count_idx2 = 0

for epoch in tqdm.tqdm(range(300)):
    
    cost = 0.0
    model.train()
    
    for batch in train_dataloader:
        count_idx +=1
        
        img = batch[0]
        wqi =batch[1]
        label = batch[2]
        
        label = label.to(torch.float32)
        label = label.to(device)
        
        x_= torch.stack(wqi).to(device)
        x_ = torch.transpose(x_,0,1)
        x_ = x_.to(torch.float32)

        output = model(x_)
        output = output.view(-1)

        loss = criterion(output, label)
        
        loss = loss.to(torch.float32)
        loss.backward()
        
        optimizer.step()
        #scheduler.step(loss)
        
        #print('Loss: ', loss, 'epoch: ', epoch, "lr: ", optimizer.param_groups[0]['lr'])
        #print('output: ', output[0:5])
        #print('label : ', label[0:5])
        if count_idx%30 == 1 :
            optimizer.param_groups[0]['lr'] *= 0.99
        
        if count_idx%100 ==1 and count_idx >100:

            writer.add_scalar('train_loss', loss, epoch)
            print('train:\t',loss, epoch)
            print(output)
            break
        
    with torch.no_grad():
        model.eval()
        for batch_v in validation_dataloader:
            count_idx2 +=1
            img = batch_v[0]
            wqi =batch_v[1]
            label = batch_v[2]

            label = label.to(torch.float32)
            label = label.to(device)

            x_= torch.stack(wqi).to(device)
            x_ = torch.transpose(x_,0,1)
            x_ = x_.to(torch.float32)

            output = model(x_)
            output = output.view(-1)
            
            loss = criterion(output, label)

            loss = loss.to(torch.float32)
            if count_idx2%100 ==1 and count_idx2 >100:          
                writer.add_scalar('validation_loss', loss, epoch)
                print('evaluate:\t', loss, epoch)
                print(output)
                print(label)
                break
            
            


  0%|          | 0/300 [00:00<?, ?it/s]

train:	 tensor(9930.1855, device='cuda:0', grad_fn=<MseLossBackward0>) 0
tensor([-17.8807, -13.2231, -12.9292,   1.8390, -17.0089, -15.4040, -15.9822,
        -13.8730, -15.6099, -13.0395, -16.0203, -10.5950, -15.7722, -17.4338,
        -15.9680, -13.7483], device='cuda:0', grad_fn=<ViewBackward0>)


  0%|          | 1/300 [00:38<3:13:44, 38.88s/it]

evaluate:	 tensor(8676.5361, device='cuda:0') 0
tensor([-4.7503, -3.8051, -3.9498, -3.6029], device='cuda:0')
tensor([90.9100, 88.0600, 86.4600, 90.9600], device='cuda:0')
train:	 tensor(62964.1016, device='cuda:0', grad_fn=<MseLossBackward0>) 1
tensor([-166.7610, -171.6743, -172.2168, -173.6967, -177.2820, -175.5456,
        -163.4548, -170.2329, -139.5984, -158.4074, -174.6502, -171.0094,
        -182.8379, -163.5651, -168.8565, -143.7478], device='cuda:0',
       grad_fn=<ViewBackward0>)


  1%|          | 2/300 [01:16<3:08:44, 38.00s/it]

...

evaluate:	 tensor(574.5071, device='cuda:0') 6
tensor([62.2045, 60.5797, 63.6053, 58.8675], device='cuda:0')
tensor([79.6200, 88.1700, 93.5500, 77.2200], device='cuda:0')
train:	 tensor(541951.8125, device='cuda:0', grad_fn=<MseLossBackward0>) 7
tensor([-694.1893, -663.2744, -677.5626, -689.2198, -622.0388, -709.4129,
        -619.3093, -727.4532, -718.2841, -682.3912, -656.6480, -599.5650,
        -646.4107, -669.7006, -157.2380, -666.0057], device='cuda:0',
       grad_fn=<ViewBackward0>)


  3%|▎         | 8/300 [05:18<3:14:39, 40.00s/it]

evaluate:	 tensor(458050.2500, device='cuda:0') 7
tensor([-647.4762, -626.7078, -705.0720, -307.2441], device='cuda:0')
tensor([87.8900, 80.8300, 91.0800, 88.9500], device='cuda:0')


  3%|▎         | 8/300 [05:46<3:30:33, 43.26s/it]


KeyboardInterrupt: 

In [32]:
lstm = RNNNet(input_size=8, num_classes=1, hidden_size = 8, init_weights=True).to(device)
lstm(x_)

tensor([[0.0869],
        [0.1284],
        [0.1484],
        [0.1420],
        [0.1419],
        [0.1232],
        [0.0899],
        [0.1331],
        [0.0910],
        [0.0974],
        [0.1557],
        [0.1553],
        [0.1554],
        [0.1545],
        [0.1491],
        [0.1201],
        [0.1454],
        [0.1333],
        [0.1031],
        [0.0919]], device='cuda:0', grad_fn=<AddmmBackward0>)